In [2]:
import numpy as np

In [3]:
y_train = np.genfromtxt("../dataset/y_train.csv", delimiter=",", dtype=int)
y_test = np.genfromtxt("../dataset/y_test.csv", delimiter=",", dtype=int)

print((y_train == 0).sum()/y_train.shape[0]*100, (y_train == 1).sum()/y_train.shape[0]*100, (y_train == 2).sum()/y_train.shape[0]*100)
print((y_test == 0).sum()/y_test.shape[0]*100, (y_test == 1).sum()/y_test.shape[0]*100, (y_test == 2).sum()/y_test.shape[0]*100)

29.956521739130437 33.130434782608695 36.91304347826087
30.142857142857142 34.0 35.85714285714286


In [4]:
x_train = np.genfromtxt("../dataset/x_train.csv", delimiter=",", dtype=str)
x_test = np.genfromtxt("../dataset/x_test.csv", delimiter=",", dtype=str)

In [5]:
train_labels = x_train[0]
train_data = x_train[1:].astype(int)

test_labels = x_test[0]
test_data = x_test[1:].astype(int)

In [5]:
good_index = np.where(train_labels == 'good')[0][0]
bad_index = np.where(train_labels == 'bad')[0][0]

print('indices:', good_index, bad_index)

train_data_positive = train_data[np.where(y_train == 2)]

good_count = train_data_positive[:,good_index].sum()
bad_count = train_data_positive[:,bad_index].sum()
total_count = train_data_positive.sum()

print('counts:', good_count, bad_count)
print('total number of words:', total_count)

log_good_prob = np.log(good_count/total_count)
log_bad_prob = np.log(bad_count/total_count)

print('log good prob:', log_good_prob)
print('log bad prob:', log_bad_prob)
print('ratio:', log_good_prob/log_bad_prob)

indices: 985 167
counts: 207 12
total number of words: 15068
log good prob: -4.287609775546563
log bad prob: -7.135421919023932
ratio: 0.6008908546970791


In [49]:
def T_sum(y):
    return train_data[np.where(y_train==y),:].sum()

def T(j, y):
    return train_data[np.where(y_train==y),np.where(train_labels==j)].sum()

def N(y):
    return (y_train == y).sum()

_N = y_train.shape[0]

def pi(y):
    return N(y) / _N

def theta(j, y):
    return T(j, y) / T_sum(y)



In [52]:
priors = np.clip(np.log(np.array([pi(y) for y in [0,1,2]])), a_min=1e-12, a_max=None)
conds = np.clip(np.log(np.array([[theta(j,y) for j in train_labels] for y in [0,1,2]])), a_min=1e-12, a_max=None)

/tmp/ipykernel_43829/3480172601.py:2: RuntimeWarning: divide by zero encountered in log
  conds = np.clip(np.log(np.array([[theta(j,y) for j in train_labels] for y in [0,1,2]])), a_min=1e-12, a_max=None)


array([[1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12],
       [1.e-12, 1.e-12, 1.e-12, ..., 1.e-12, 1.e-12, 1.e-12]])

In [6]:
num_classes = np.max(y_train) + 1
num_features = train_data.shape[1]
num_samples = y_train.shape[0]

counts = np.bincount(y_train, minlength=num_classes)
_N = num_samples
pi = counts / _N
pi = np.clip(pi, a_min=1e-12, a_max=None)
priors = np.log(pi)

y_train_onehot = np.zeros((num_samples, num_classes))
y_train_onehot[np.arange(num_samples), y_train] = 1
T_all = y_train_onehot.T @ train_data

T_sum_all = T_all.sum(axis=1)
T_sum_all = np.clip(T_sum_all, a_min=1e-12, a_max=None)

print(T_all.shape, T_sum_all[:, np.newaxis].shape)

theta = T_all / T_sum_all[:, np.newaxis]
theta = np.clip(theta, a_min=1e-12, a_max=None)
conds = np.log(theta)

(3, 3000) (3, 1)


In [7]:
predicted_label = np.argmax(priors[np.newaxis, :] + test_data @ conds.T, axis=1)

accuracy = np.mean(predicted_label == y_test)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 58.29%


In [11]:
num_classes = np.max(y_train) + 1
num_features = train_data.shape[1]
num_samples = y_train.shape[0]

counts = np.bincount(y_train, minlength=num_classes)
_N = num_samples
pi = counts / _N
#pi = np.clip(pi, a_min=1e-12, a_max=None)
priors = np.log(pi)

y_train_onehot = np.zeros((num_samples, num_classes))
y_train_onehot[np.arange(num_samples), y_train] = 1
T_all = y_train_onehot.T @ train_data

T_sum_all = T_all.sum(axis=1)
#T_sum_all = np.clip(T_sum_all, a_min=1e-12, a_max=None)

alpha = 1
theta = (T_all + alpha) / (T_sum_all[:, np.newaxis] + alpha * num_features)
#theta = np.clip(theta, a_min=1e-12, a_max=None)
conds = np.log(theta)

0


In [9]:
predicted_label = np.argmax(priors[np.newaxis, :] + test_data @ conds.T, axis=1)

accuracy = np.mean(predicted_label == y_test)

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 64.86%
